<a href="https://colab.research.google.com/github/Rossel/Digital_Music_Store/blob/master/Chinook_Music_Store.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Chinook Music Store**

## 1. Introduction

In this notebook I analyse a SQL database for a *fictional* digital music shop called Chinook, which can be compared to the iTunes Music store. Its management wants to obtain insights into its sales performance to further optimize its business strategy. The central question being asked by Chinook's management is:

**How can Chinook improve its performance and optimize its sales strategy?**


To answer this question, and various sub-questions, this report will cover the following topics:

* Best selling product categories
* Sales performance per country
* Sales agents performance
* Customer base
* Music inventory


###The database

For this report, a modified version of an SQLite database is being used from [this resource](https://github.com/lerocha/chinook-database). The actual version used in this report can be found [here](https://drive.google.com/file/d/1edlG-tOLPxLEWP19dBmBp_NxRmc7GAcx/view?usp=sharing). It contains information about the artists, songs, and albums from the music shop, as well as information on the shop's employees, customers, and the customer's purchases. This information is contained in eleven tables. Below is a schema diagram for the Chinook database:

![alt text](https://s3.amazonaws.com/dq-content/189/chinook-schema.svg)


### Importing the database

This notebook is written in Google Colaboratory, which can be likened to a Jupyter Notebook within the Google Suite. It allows users to write notes, run code and visualise data within the same document. In the steps below `chinook.db` will be loaded in this Google Collaboratory notebook. 


In [ ]:
# Import functions from Google modules into Colaboratory
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
# Insert file id from Google Drive shareable link:
# https://drive.google.com/file/d/1edlG-tOLPxLEWP19dBmBp_NxRmc7GAcx/view?usp=sharing
id = '1edlG-tOLPxLEWP19dBmBp_NxRmc7GAcx'

In [ ]:
# Download the dataset
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('chinook.db')

In [ ]:
# Import SQLite3 and pandas library
import sqlite3
import pandas as pd

In [ ]:
# Connect the notebook to the database file
%%capture
%load_ext sql
%sql sqlite:///chinook.db

To verify the database is fully functional, a query is run to return information on the tables and views in the database.

In order to run SQL queries in a Collab notebook, the IPython Magic Command `%%sql` needs to be run on the first line of every cell, followed by the SQL query itself. 

In [ ]:
%%sql

SELECT name, type
  FROM sqlite_master
 WHERE type IN ("table", "view");

 * sqlite:///chinook.db
Done.


name,type
album,table
artist,table
customer,table
employee,table
genre,table
invoice,table
invoice_line,table
media_type,table
playlist,table
playlist_track,table


## 2. Best selling product categories

In this section, the following questions will be answered:
* Who are the best selling artists?
* What are the best selling albums? 
* What are the best selling genres?
* Which artists are most sold through playlists?

### Best selling artists

In [ ]:
%%sql

SELECT
    ta.artist_name artist,
    COUNT(*) tracks_purchased
FROM invoice_line il
INNER JOIN (
            SELECT
                t.track_id,
                ar.name artist_name
            FROM track t
            INNER JOIN album al ON al.album_id = t.album_id
            INNER JOIN artist ar ON ar.artist_id = al.artist_id
           ) ta
           ON ta.track_id = il.track_id
GROUP BY 1
ORDER BY 2 DESC 
LIMIT 10;

 * sqlite:///chinook.db
Done.


artist,tracks_purchased
Queen,192
Jimi Hendrix,187
Nirvana,130
Red Hot Chili Peppers,130
Pearl Jam,129
AC/DC,124
Guns N' Roses,124
Foo Fighters,121
The Rolling Stones,117
Metallica,106


**Queen** and **Jimi Hendrix** are the most popular artists in the Chinook music store.

Using the query above in reversed order, the least popular artists can be found that could be eliminated from the database to cut costs for Chinook. Chinooks management would have to decide before proceeding with this, so currently it remains outside the scope of this project. A date clause would have to be included to avoid elimination of artists that have recently been added to the inventory.

### Best selling albums

In [ ]:
%%sql

SELECT
    ta.album_title album,
    ta.artist_name artist,
    COUNT(*) tracks_purchased
  FROM invoice_line il
 INNER JOIN (
            SELECT
                t.track_id,
                al.title album_title,
                ar.name artist_name
            FROM track t
            INNER JOIN album al ON al.album_id = t.album_id
            INNER JOIN artist ar ON ar.artist_id = al.artist_id
           ) ta
           ON ta.track_id = il.track_id
 GROUP BY 1, 2
 ORDER BY 3 DESC 
 LIMIT 5;

 * sqlite:///chinook.db
Done.


album,artist,tracks_purchased
Are You Experienced?,Jimi Hendrix,187
Faceless,Godsmack,96
Mezmerize,System Of A Down,93
Get Born,JET,90
The Doors,The Doors,83


The album "**Are You Experienced?**" by **Jimi Hendrix** is sold most according to the database. Below is an overview of the tracks on this album.

In [ ]:
%%sql

SELECT * FROM
    (
     SELECT
         t.name,
         ar.name artist,
         al.title album_name,
         mt.name media_type,
         g.name genre,
         t.milliseconds length_milliseconds
     FROM track t
     INNER JOIN media_type mt ON mt.media_type_id = t.media_type_id
     INNER JOIN genre g ON g.genre_id = t.genre_id
     INNER JOIN album al ON al.album_id = t.album_id
     INNER JOIN artist ar ON ar.artist_id = al.artist_id
    )
WHERE album_name = "Are You Experienced?";

 * sqlite:///chinook.db
Done.


name,artist,album_name,media_type,genre,length_milliseconds
Foxy Lady,Jimi Hendrix,Are You Experienced?,MPEG audio file,Rock,199340
Manic Depression,Jimi Hendrix,Are You Experienced?,MPEG audio file,Rock,222302
Red House,Jimi Hendrix,Are You Experienced?,MPEG audio file,Rock,224130
Can You See Me,Jimi Hendrix,Are You Experienced?,MPEG audio file,Rock,153077
Love Or Confusion,Jimi Hendrix,Are You Experienced?,MPEG audio file,Rock,193123
I Don't Live Today,Jimi Hendrix,Are You Experienced?,MPEG audio file,Rock,235311
May This Be Love,Jimi Hendrix,Are You Experienced?,MPEG audio file,Rock,191216
Fire,Jimi Hendrix,Are You Experienced?,MPEG audio file,Rock,164989
Third Stone From The Sun,Jimi Hendrix,Are You Experienced?,MPEG audio file,Rock,404453
Remember,Jimi Hendrix,Are You Experienced?,MPEG audio file,Rock,168150


![Jimi Hendrix](https://1.bp.blogspot.com/-_KAun5P_CU0/UI9S8RSQ7eI/AAAAAAAAAhc/K2XrzWc2_rY/s1600/Jimi+Hendrix.jpg)

### Best selling genres

In [ ]:
%%sql 

SELECT 
       g.name genre_name,
       SUM(il.quantity) tracks_sold FROM track t
 INNER JOIN invoice_line il ON il.track_id = t.track_id INNER JOIN genre g ON g.genre_id = t.genre_id
 GROUP BY 1
 ORDER BY 2 DESC;

 * sqlite:///chinook.db
Done.


genre_name,tracks_sold
Rock,2635
Metal,619
Alternative & Punk,492
Latin,167
R&B/Soul,159
Blues,124
Jazz,121
Alternative,117
Easy Listening,74
Pop,63


**Rock** is by far the best selling genre on Chinook, followed by **Metal** and **Alternative & Punk**. It is advisable for Chinook to leverage its strength in these genres, and invest to expand its inventory in these genres. In its marketing expressions, Chinook should profile itself as a **specialist** in these genres to further attrack audiences interested in these genres.

The opposite, more **generalist** approach of boosting sales efforts in weaker genres (such as Soundtracks, Hip Hop/Rap and Reggae) could expand the user base substantially, but might be less cost efficient. 

[**The Long Tail: Why the Future of Business is Selling Less of More**](https://www.amazon.com/Long-Tail-Future-Business-Selling/dp/1401309666) by Chris Anderson describes why this generalist might pay off in the long term, but further marketing research into this topic would have to be done. Another important phenomenon to study is the direction of technological development, such as the current dominance of streaming music services.

## 3. Sales performance per country

This section will answer the following question: 

**What countries generate most sales, and which the least?**

In [ ]:
%%sql
WITH cust_country AS
                (
                    SELECT c.customer_id,
                            c.country ,
                            COUNT(i.invoice_id) no_of_purchases,
                            SUM(i.total) total_purchase_per_cust
                    FROM customer c 
                    INNER JOIN invoice i ON i.customer_id = c.customer_id
                    GROUP BY 1,2
                    ORDER BY 2
                ),
    country_group AS
                (
                    SELECT cc.country,
                           COUNT(cc.customer_id) total_customers,
                           SUM(cc.total_purchase_per_cust) Total_sales,
                            SUM(cc.total_purchase_per_cust)/SUM(cc.no_of_purchases) AVG_order_value,
        
                            CASE 
                                WHEN COUNT(cc.customer_id) = 1 THEN 'Other'
                                ELSE cc.country   
                            END AS country1
        
           
             
                    FROM cust_country cc
                    GROUP BY 1
                ),
        country_sort AS
                    (
                        SELECT 
                            cg.country1 country,
                            SUM(cg.total_customers) Total_customers,
                            SUM(cg.Total_sales) Total_sales,
                            SUM(cg.Total_sales)/SUM(cg.Total_customers) AVG_Sales_Per_Customer,
                            cg.AVG_order_value Avg_order_value,
                            CASE
                                WHEN cg.country1='Other' THEN 1
                                ELSE 0
                            END AS sort 
                        FROM country_group cg
                        GROUP BY 1
                        ORDER BY sort ASC ,Total_sales DESC
                    )
SELECT cs.country Country,
       cs.Total_customers Total_customers,
       ROUND(cs.Total_sales,2) Total_sales,
    ROUND(cs.AVG_sales_Per_Customer,2) AVG_sales_Per_Customer,
       ROUND(cs. Avg_order_value,2) AVG_order_value
FROM country_sort cs

 * sqlite:///chinook.db
Done.


Country,Total_customers,Total_sales,AVG_sales_Per_Customer,AVG_order_value
USA,13,1040.49,80.04,7.94
Canada,8,535.59,66.95,7.05
Brazil,5,427.68,85.54,7.01
France,5,389.07,77.81,7.78
Germany,4,334.62,83.66,8.16
Czech Republic,2,273.24,136.62,9.11
United Kingdom,3,245.52,81.84,8.77
Portugal,2,185.13,92.56,6.38
India,2,183.15,91.57,8.72
Other,15,1094.94,73.0,7.52


The **USA** generates the most sales, followed by **Canada**, while average sales per customer are the lowest for these countries. The **Czech Republic** has the highest average sale per customer and the highest average order value. It would be advisable to concentrate the marketing efforts in these countries. The countries with the lowest sales are **Portugal** and **India**. 

## 4. Sales agents performance

This section will answer the question: **How do the sales support agents perform?**

Below is an overview of Chinook's employees and its reporting structure. There are three Sales Support Agents who report to Nancy Edwards, who is the Sales Manager at Chinook. The names of the Sales Support Agents are:
* Jane Peacock
* Margaret Park
* Steve Johnson

In [ ]:
%%sql

SELECT
    e1.first_name || " " || e1.last_name employee_name,
    e1.title employee_title,
    e2.first_name || " " || e2.last_name supervisor_name,
    e2.title supervisor_title
FROM employee e1
LEFT JOIN employee e2 ON e1.reports_to = e2.employee_id
ORDER BY 1;

 * sqlite:///chinook.db
Done.


employee_name,employee_title,supervisor_name,supervisor_title
Andrew Adams,General Manager,None,None
Jane Peacock,Sales Support Agent,Nancy Edwards,Sales Manager
Laura Callahan,IT Staff,Michael Mitchell,IT Manager
Margaret Park,Sales Support Agent,Nancy Edwards,Sales Manager
Michael Mitchell,IT Manager,Andrew Adams,General Manager
Nancy Edwards,Sales Manager,Andrew Adams,General Manager
Robert King,IT Staff,Michael Mitchell,IT Manager
Steve Johnson,Sales Support Agent,Nancy Edwards,Sales Manager


Each customer for the Chinook store gets assigned to a sales support agent within the company when they first make a purchase. Below is a breakdown of the performance of each sales support agent. Additional columns with information on the agents are included, to explain any variance that might be influencing their sales performance.

In [ ]:
%%sql

WITH total_purchase_per_cust AS
            (
                SELECT c.support_rep_id ,
                        c.customer_id cust_id,
                        SUM(i.total) total_per_customer
                FROM customer c
                INNER JOIN invoice i ON i.customer_id=c.customer_id
                GROUP BY 1, 2
                ORDER By 1
            ),
    total_sales_per_emp AS
                (
                    SELECT tpc.support_rep_id,
                            SUM(tpc.total_per_customer) total_sales_by_emp
                    
                    FROM total_purchase_per_cust tpc
                     GROUP BY 1
                            
                )
SELECT 
    e.employee_id,
    e.first_name||" "||e.last_name employee_name,
    round(tse.total_sales_by_emp,3) total_sales_by_employee,
    e.hire_date,
    e.city,
    e.birthdate
FROM employee e
INNER JOIN total_sales_per_emp tse ON tse.support_rep_id=e.employee_id

ORDER BY 1,2 DESC

 * sqlite:///chinook.db
Done.


employee_id,employee_name,total_sales_by_employee,hire_date,city,birthdate
3,Jane Peacock,1731.51,2017-04-01 00:00:00,Calgary,1973-08-29 00:00:00
4,Margaret Park,1584.0,2017-05-03 00:00:00,Calgary,1947-09-19 00:00:00
5,Steve Johnson,1393.92,2017-10-17 00:00:00,Calgary,1965-03-03 00:00:00


We can see that **Jane Peacock** achieved the highest sales, while Steve Johnson has the lowest performance. There is about 20% difference in sales turnover between the two Sales Support Agents. Steve does have about 6 months fewer work experience in the position than his colleagues, which could be an explanation. Jane is the youngest of the three, which may explain why she connects better to a younger audience. It might be worth investigating how to stimulate the sales team, for instance by optimizing the commission scheme.

![Queen album cover](https://i.pinimg.com/originals/35/68/65/3568656e701dd36aaa57c29f228b7e1b.jpg)

## 5. Customer base

In this section, the customer base is analysed at a deeper level. It will answer the following questions:
* Which customers have spent over USD 90?
* Who are the best customers per country?
* Who are the best customers in the USA?

The first question is answered below:

In [ ]:
%%sql

CREATE VIEW customer_gt_90_dollars AS 
    SELECT
        c.*
    FROM invoice i
    INNER JOIN customer c ON i.customer_id = c.customer_id
    GROUP BY 1
    HAVING SUM(i.total) > 90;
SELECT * FROM customer_gt_90_dollars;

 * sqlite:///chinook.db
Done.
Done.


customer_id,first_name,last_name,company,address,city,state,country,postal_code,phone,fax,email,support_rep_id
1,Luís,Gonçalves,Embraer - Empresa Brasileira de Aeronáutica S.A.,"Av. Brigadeiro Faria Lima, 2170",São José dos Campos,SP,Brazil,12227-000,+55 (12) 3923-5555,+55 (12) 3923-5566,luisg@embraer.com.br,3
3,François,Tremblay,None,1498 rue Bélanger,Montréal,QC,Canada,H2G 1A7,+1 (514) 721-4711,None,ftremblay@gmail.com,3
5,František,Wichterlová,JetBrains s.r.o.,Klanova 9/506,Prague,None,Czech Republic,14700,+420 2 4172 5555,+420 2 4172 5555,frantisekw@jetbrains.com,4
6,Helena,Holý,None,Rilská 3174/6,Prague,None,Czech Republic,14300,+420 2 4177 0449,None,hholy@gmail.com,5
13,Fernanda,Ramos,None,Qe 7 Bloco G,Brasília,DF,Brazil,71020-677,+55 (61) 3363-5547,+55 (61) 3363-7855,fernadaramos4@uol.com.br,4
17,Jack,Smith,Microsoft Corporation,1 Microsoft Way,Redmond,WA,USA,98052-8300,+1 (425) 882-8080,+1 (425) 882-8081,jacksmith@microsoft.com,5
20,Dan,Miller,None,541 Del Medio Avenue,Mountain View,CA,USA,94040-111,+1 (650) 644-3358,None,dmiller@comcast.com,4
21,Kathy,Chase,None,801 W 4th Street,Reno,NV,USA,89503,+1 (775) 223-7665,None,kachase@hotmail.com,5
22,Heather,Leacock,None,120 S Orange Ave,Orlando,FL,USA,32801,+1 (407) 999-7788,None,hleacock@gmail.com,4
30,Edward,Francis,None,230 Elgin Street,Ottawa,ON,Canada,K2P 1L7,+1 (613) 234-3322,None,edfrancis@yachoo.ca,3


Customers that spend over USD 90 are of **high value to Chinook** and for this reason, it is important to maintain warm relations with these individuals. They can be marketed directly with exclusive offers and promotions. 

Below follows the answer to the next question:

In [ ]:
%%sql

WITH
    customer_country_purchases AS
        (
         SELECT
             i.customer_id,
             c.country,
             ROUND(SUM(i.total),2) total_purchases
         FROM invoice i
         INNER JOIN customer c ON i.customer_id = c.customer_id
         GROUP BY 1, 2
        ),
    country_max_purchase AS
        (
         SELECT
             country,
             MAX(total_purchases) max_purchase
         FROM customer_country_purchases
         GROUP BY 1
        ),
    country_best_customer AS
        (
         SELECT
            cmp.country,
            cmp.max_purchase,
            (
             SELECT ccp.customer_id
             FROM customer_country_purchases ccp
             WHERE ccp.country = cmp.country AND cmp.max_purchase = ccp.total_purchases
            ) customer_id
         FROM country_max_purchase cmp
        )
SELECT
    cbc.country country,
    c.first_name || " " || c.last_name customer_name,
    cbc.max_purchase total_purchased
FROM customer c
INNER JOIN country_best_customer cbc ON cbc.customer_id = c.customer_id
ORDER BY 1 ASC

 * sqlite:///chinook.db
Done.


country,customer_name,total_purchased
Argentina,Diego Gutiérrez,39.6
Australia,Mark Taylor,81.18
Austria,Astrid Gruber,69.3
Belgium,Daan Peeters,60.39
Brazil,Luís Gonçalves,108.9
Canada,François Tremblay,99.99
Chile,Luis Rojas,97.02
Czech Republic,František Wichterlová,144.54
Denmark,Kara Nielsen,37.62
Finland,Terhi Hämäläinen,79.2


These are the best customers per country. As mentioned above, all customers that spend over USD 90 on Chinook are considered of **high value** and can be targeted with **direct marketing**, based on their previous purchases and preferences.

As demonstrated earlier, the USA is the primary market for Chinook. Below is an overview of the most valuable customers for this country.

In [ ]:
%%sql

WITH
    customers_usa AS
        (
        SELECT * FROM customer
        WHERE country = "USA"
        ),
    sales_per_customer AS
        (
         SELECT
             customer_id,
             ROUND(SUM(total),2) total
         FROM invoice
         GROUP BY 1
        )

SELECT
    cu.first_name || " " || cu.last_name customer_name,
    spc.total total_purchases
FROM customers_usa cu
INNER JOIN sales_per_customer spc ON cu.customer_id = spc.customer_id
ORDER BY 2 DESC;

 * sqlite:///chinook.db
Done.


customer_name,total_purchases
Jack Smith,98.01
Dan Miller,95.04
Heather Leacock,92.07
Kathy Chase,91.08
Richard Cunningham,86.13
Patrick Gray,84.15
Michelle Brooks,79.2
Victor Stevens,76.23
Frank Harris,74.25
Julia Barnett,72.27


It is advisable for the Sales Support Agents to consider all-high value customers in both the USA and other countries, to better understand their preferences, maintain good customer relations and avoid losing them to the competition.

![The Doors album art](https://images-na.ssl-images-amazon.com/images/I/91hrbype4aL._SL1500_.jpg)

## 6. Music inventory

In this final section, we analyse the inventory (or library) of tracks that Chinook offers. We will answer the following questions:
* Do customers mostly buy individual tracks or full albums?
* Does the entire inventory get sold? What share remains unsold?
* Does protected make up a large share of total sales?

The first question will be answered below. It is of importance, because currently Chinook purchases entire albums from record companies, which is less cost efficient than buying only individual tracks. 

In [ ]:
%%sql
WITH invoice_info AS
                  (
                    SELECT il.invoice_id invoice_id,
                              il.track_id track_id,
                              t.album_id album_id
                    FROM invoice_line il
                    INNER JOIN track t ON t.track_id = il.track_id
                  ),

    Album_info AS
                (
                  SELECT ii.invoice_id ,
                            ii.track_id,
                            ii.album_id
                  FROM invoice_info ii
                  GROUP BY 1
                ),

    Album_or_track AS
                    (
                     SELECT ai.invoice_id ,
                        CASE
                           WHEN
                                (
                                  (SELECT t.track_id
                                    FROM track t
                                    WHERE t.album_id = ai.album_id
                                    EXCEPT
                                    SELECT ii.track_id 
                                    FROM invoice_info ii
                                    WHERE ii.invoice_id = ai.invoice_id
                                    )IS NULL
                                    
                                  AND
                                  (SELECT ii.track_id 
                                    FROM invoice_info ii
                                    WHERE ii.invoice_id = ai.invoice_id
                                    EXCEPT
                                    SELECT t.track_id
                                    FROM track t
                                    WHERE t.album_id = ai.album_id
                                    )IS NULL
                                   )THEN "YES"
                                    ELSE "NO"
                                END AS ALBUM_Purchased
                        FROM Album_info ai
                    )

SELECT  COUNT(at.invoice_id) No_of_invoices,
       ROUND(CAST(COUNT(at.invoice_id)AS FLOAT)*100/(SELECT COUNT(*) FROM invoice),2) Percentage,
        at.ALBUM_purchased album_purchased
        
FROM Album_or_track at
GROUP BY 3;

 * sqlite:///chinook.db
Done.


No_of_invoices,Percentage,album_purchased
500,81.43,NO
114,18.57,YES


 The percentage of **full albums purchased is only 18.57%**, which makes it inefficient for Chinook to continue buying these from record labels. It is worth considering the purchase of only the most popular tracks.

Below, the second question will be answered to determine what share of the total inventory remains unsold. 

In [ ]:
%%sql

WITH track_table AS
              (
                  SELECT t.track_id
                  FROM track t
              ),
    purchase_info AS
                  ( 
                      SELECT tt.track_id,
                      CASE 
                         WHEN 
                          (
                              tt.track_id 
                              IN
                              (SELECT il.track_id
                              FROM invoice_line il)
                               
                           )THEN "Sold"
                           ELSE "Unsold" 
                        END AS purchase_or_not 
                      FROM track_table tt
                       )
        
SELECT pi.purchase_or_not,
       COUNT(pi.track_id) No_of_tracks,
       ROUND(CAST(COUNT(pi.track_id)AS FLOAT)*100/(SELECT COUNT(*)FROM track t),2) Percentage_of_Purchase
 FROM purchase_info pi
GROUP BY 1;

 * sqlite:///chinook.db
Done.


purchase_or_not,No_of_tracks,Percentage_of_Purchase
Sold,1806,51.56
Unsold,1697,48.44


**Almost half** of the entire inventory remains **unsold**, while the other has been sold at least once. A wide choice and availability of tracks is what customers might value, but it comes at a high cost for Chinook. As demonstrated in the previous query, it would be more efficient to purchase single tracks from record labels and focus on the most popular tracks and artists.

The final question will be answered below, where it is investigated what the impact is of protected media types. There are five different types of media offered on Chinook, of which two are protected file types. 

In [ ]:
%%sql
SELECT *
FROM media_type

 * sqlite:///chinook.db
Done.


media_type_id,name
1,MPEG audio file
2,Protected AAC audio file
3,Protected MPEG-4 video file
4,Purchased AAC audio file
5,AAC audio file


The other three file types will be classified as Non_protected.

In [ ]:
%%sql
WITH in_store_track AS
                   (
                    SELECT COUNT(t.track_id) tracks_in_store,
                               t.media_type_id,
                                mt.name
                    FROM track t
                    INNER JOIN media_type mt ON mt.media_type_id = t.media_type_id
                    GROUP BY 3
                   ),
    track_sold AS
                (
                 SELECT COUNT(il.track_id) tracks_sold,
                         t.media_type_id,
                         mt.name
                 FROM invoice_line il
                 INNER JOIN track t ON t.track_id = il.track_id
                 INNER JOIN media_type mt ON mt.media_type_id = t.media_type_id
                 GROUP BY 3
                ),
        
    store_vs_sold AS
                (
                 SELECT it.name media_type,
                           it.tracks_in_store,
                           ts.tracks_sold
                 FROM in_store_track it
                 INNER JOIN track_sold ts On ts.media_type_id = it.media_type_id
                ),
        protected_vs_nonprotected AS
                              (
                                SELECT SUM(ss.tracks_in_store) tracks_in_store,
                                       SUM(ss.tracks_sold) tracks_sold,
                                       CASE 
                                          WHEN
                                            (  
                                              ss.media_type like "%Protected%"
                                             )THEN "Protected"
                                             ELSE  "Non_protected"
                                      END AS Media_type
                                FROM store_vs_sold ss
                                GROUP BY 3
                              )

SELECT pn.Media_type,
       pn.tracks_in_store,
       ROUND(CAST(pn.tracks_in_store AS FLOAT)*100/(SELECT COUNT(t.track_id) FROM track t),2) PERCENTAGE_TRACKS_in_STORE,
       pn.tracks_sold,
       ROUND(CAST(pn.tracks_sold AS FLOAT)*100/(SELECT COUNT(il.track_id) FROM invoice_line il),2) PERCENTAGE_TRACKS_SOLD
FROM protected_vs_nonprotected pn

 * sqlite:///chinook.db
Done.


Media_type,tracks_in_store,PERCENTAGE_TRACKS_in_STORE,tracks_sold,PERCENTAGE_TRACKS_SOLD
Non_protected,3052,87.13,4315,90.71
Protected,451,12.87,442,9.29


The percentage of protected tracks sold is only 9.29%, while it makes up for 12.87% of tracks in the inventory. This means that protected media types are lower in demand than non-protected media. It is worth considering to only offer non-protected media types.



---



## 7. Conclusions

From the analysis of Chinook's database, we can conclude the following points:
- Best selling product categories: **Rock music** is the speciality of Chinook, of which it sells the majority of its tracks. The top 5 best selling albums fall in this category, so it is worth considering to emphasize this in Chinook's marketing and profiling.
- Sales performance per country: the **USA** is by far the largest market for Chinook. Chinook's marketing efforts should be aimed at strong holding its position in this market.
- Sales Agents performance: There is a considerable difference in sales turnover between Sales Support Agents, with about 20% difference between the best and worst performer. It might be worth investigating how to improve **sales team motivation**, for instance by optimizing the commission scheme.
- Customer base: **high-value customers** could be personally targeted with **direct marketing**, based on their previous purchases. Sales Support Agents should be motivated to better understand their **preferences**, maintain good customer **relations** and avoid losing them to the **competition**.
- Music inventory: The percentage of full albums purchased is only 18.57%. From a cost reduction perspective, it can be interesting to **purchase only the most popular tracks** in **non-protected media form** from record companies. Another reason for this is that about half of the full music inventory remains unsold. A wide choice and **availability of tracks** may be what customers value, although it may come at a high expense for Chinook. Rapid technological developments, such as the growing dominance of music streaming services need to be monitored and included in Chinook's strategy plans. It would be advisable to make clear decisions with regards to its **purchasing policies**.


---